### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [3]:
unique_players = purchase_data.loc[:, ['SN']]
unique_players = unique_players.drop_duplicates()
total_player = unique_players.count()[0]  #get all the unique values first, using SN at this point

pd.DataFrame({'Total Players': [total_player]})

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
item_id = purchase_data['Item ID'].value_counts() #get Number of Unique items via "Item ID"
#print(item_id.shape[0])

total_revenue = purchase_data['Price'].sum() #sum of price to get revenue
#print(total_revenue)
average_price = round(purchase_data['Price'].mean() , 2) #mean of price to get average price
#print(average_price)

num_purchases = purchase_data['Purchase ID'].value_counts() #get number of purchases via "Purchase ID"
#print(num_purchases.shape[0])

purchase_analysis = {'Number of Unique Items':[item_id.shape[0]], 'Average Price': [average_price],
                     'Number of Purchases':[num_purchases.shape[0]], 'Total Revenue': [total_revenue] }


purchasing_analysis_df = pd.DataFrame(purchase_analysis, index=[0]) #creating a new DF on purchase_analysis dictionary

purchasing_analysis_df['Average Price'] = purchasing_analysis_df['Average Price'].map('${:.2f}'.format)
purchasing_analysis_df['Total Revenue'] = purchasing_analysis_df['Total Revenue'].map('${:.2f}'.format)
# pd.options.display.float_format = '${:,.2f}'.format - cannot be used in this case
# as it will make all float with the same format

purchasing_analysis_df




,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


Looking at the above DataFrame, the game Heroes of Pymoli currently have 576 players with 780 purchase transactions, meaning some players are making reoccurring in-game purchases. 

## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [23]:
gender_count = purchase_data.groupby(['Gender']) #group purchase data by Gender

player_gender = gender_count.nunique()['SN'] #to get unique value for gender using SN as SN is unique
#print (player_gender) to check values, total should be 576

player_perc = player_gender / total_player * 100
#print(player_perc) to check if calculations is correct

#creating te DataFrame to put all the data together
gender_demo_df = pd.DataFrame({'Total Players' : player_gender,
                               'Percentage of Players': player_perc} , index=['Male','Female','Other / Non-Disclosed'])

gender_demo_df['Percentage of Players'] = gender_demo_df['Percentage of Players'].map('{:.2f}%'.format)
#formatting to have Percentage in 2 decimals and % at the end. 
gender_demo_df


,Total Players,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


The players in the game are predominantly Male (> 84%).


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#gender_display is unique and can also be used as purchase count, or can also use Purchase ID.
#alternatively - purchase_analysis_gender = purchase_data.groupby(['Gender'])['Price'].count()

#getting average purchase price for each gender, initially using the following however its too repetitive and long winded
# average_purchase_female= purchase_data.loc[(purchase_data.Gender == 'Female'), 'Price'].mean()
# average_purchase_male = purchase_data.loc[(purchase_data.Gender == 'Male'), 'Price'].mean()
# average_purchase_nondis = purchase_data.loc[(purchase_data.Gender == 'Other / Non-Disclosed'), 'Price'].mean()
average_purchase_gender = purchase_data.groupby(['Gender'])['Price'].mean()

#getting total purchase value for each gender, similarly with the above, using groupby method, much simpler
# total_purchase_female = purchase_data.loc[(purchase_data.Gender == 'Female'), 'Price'].sum()
# total_purchase_male = purchase_data.loc[(purchase_data.Gender == 'Male'), 'Price'].sum()
# total_purchase_nondis = purchase_data.loc[(purchase_data.Gender == 'Other / Non-Disclosed'), 'Price'].sum()
total_purchase_gender = purchase_data.groupby(['Gender'])['Price'].sum()

#getting average total purchase per gender with total purchase value divided by each gender demographics
average_totpur_gender = total_purchase_gender/gender_demo_df['Total Players']


#let's create the DataFrame
purchase_analysis_gender_df = pd.DataFrame({'Purchase Count' : gender_display,
                                           'Average Purchase Price': average_purchase_gender,
                                           'Total Purchase Value': total_purchase_gender,
                                           'AVG Total Purchase per Person': average_totpur_gender })
#format money with $
purchase_analysis_gender_df['Average Purchase Price'] = purchase_analysis_gender_df['Average Purchase Price'].map('${:.2f}'.format)
purchase_analysis_gender_df['Total Purchase Value'] = purchase_analysis_gender_df['Total Purchase Value'].map('${:.2f}'.format)
purchase_analysis_gender_df['AVG Total Purchase per Person'] = purchase_analysis_gender_df['AVG Total Purchase per Person'].map('${:.2f}'.format)

purchase_analysis_gender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,AVG Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$3.20
Male,652,$3.02,$1967.64,$3.02
Other / Non-Disclosed,15,$3.35,$50.19,$3.35


Male players are also the biggest spender in the game contributing to a Total Purchase value of $1967.64. 

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [33]:
# Create the bins in which Data will be held 
bins = [0, 9.99, 14.99, 19.99, 24.99, 29.99, 34.99, 39.99, 999]

# Create the names for the five bins
age_bin_labels = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

#using pd.cut to create a new column 'Age Range' in the original purchase_data
purchase_data["Age Range"] = pd.cut(purchase_data["Age"], bins, labels= age_bin_labels, include_lowest=True)
age_grouped = purchase_data.groupby("Age Range") 
age_range_total = age_grouped["SN"].nunique()  #to get individual count for each age range
age_perc = (age_range_total / total_player)* 100 #to get percentage of age group

#let's create the DataFrame
age_demo_analysis_df = pd.DataFrame({'Total Count': age_range_total,
                                   'Percentage of Players': age_perc})

#formatting, to have Percentage of Players with % and rearrange the index
age_demo_analysis_df['Percentage of Players'] = age_demo_analysis_df['Percentage of Players'].map('{:.2f}%'.format)
age_demo_analysis_df.sort_index() # this is required as DataFrame is showing Total count as descending 


,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


Most players fall in the age range of 15 to 29 with concentration in the 20 to 24 range.

## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [26]:
#re-using previous 'Age Range' bins
#getting all required data using groupby and count mean sum trio
purchase_count = purchase_data.groupby(['Age Range'])['Price'].count()
avg_purprice = purchase_data.groupby(['Age Range'])['Price'].mean()
total_purvalue = purchase_data.groupby(['Age Range'])['Price'].sum()
avg_totalppp = total_purvalue / purchase_count

#creating the DataFrame

purchasing_aa = pd.DataFrame({'Purchase Count': purchase_count,
                             'Average Purchase Price': avg_purprice,
                             'Total Purchase Value': total_purvalue,
                             'Avg Total Purchase per Person': avg_totalppp})
#formatting, all money data with $
purchasing_aa['Average Purchase Price'] = purchasing_aa['Average Purchase Price'].map('${:.2f}'.format)
purchasing_aa['Total Purchase Value'] = purchasing_aa['Total Purchase Value'].map('${:.2f}'.format)
purchasing_aa['Avg Total Purchase per Person'] = purchasing_aa['Avg Total Purchase per Person'].map('${:.2f}'.format)

purchasing_aa

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
<10,23,$3.35,$77.13,$3.35
10-14,28,$2.96,$82.78,$2.96
15-19,136,$3.04,$412.89,$3.04
20-24,365,$3.05,$1114.06,$3.05
25-29,101,$2.90,$293.00,$2.90
30-34,73,$2.93,$214.00,$2.93
35-39,41,$3.60,$147.67,$3.60
40+,13,$2.94,$38.24,$2.94


20 - 24 being the largest age group made the most in-game purchases

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
# to use the count mean sum trio again together with groupby 'SN' get the values for the DataFrame 
ind_purchase = purchase_data.groupby(['SN'])['Price'].count()
#ind_purchase.sort_values(ascending=False) use to get descending value
ind_avg_purchase = purchase_data.groupby(['SN'])['Price'].mean()
ind_tot_purchase = purchase_data.groupby(['SN'])['Price'].sum()

#creating the DataFrame

top_spenders = pd.DataFrame({'Purchase Count': ind_purchase,
                            'Average Purchase Price': ind_avg_purchase,
                            'Total Purchase Value': ind_tot_purchase,
                             'Avg Total Purchase per Person': ind_tot_purchase})

#formatting, all money data with $ and to sort DataFrame to display in descending order to get top spenders on top
top_spenders['Average Purchase Price'] = top_spenders['Average Purchase Price'].map('${:.2f}'.format)
top_spenders['Total Purchase Value'] = top_spenders['Total Purchase Value'].map('${:.2f}'.format)
top_spenders['Avg Total Purchase per Person'] = top_spenders['Avg Total Purchase per Person'].map('${:.2f}'.format)
top_spenders.sort_values(by=['Purchase Count'],ascending = False).head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
SN,,,,
Lisosia93,5,$3.79,$18.96,$18.96
Iral74,4,$3.40,$13.62,$13.62
Idastidru52,4,$3.86,$15.45,$15.45
Asur53,3,$2.48,$7.44,$7.44
Inguron55,3,$3.70,$11.11,$11.11


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
# to use the count and sum with groupby 'Item ID' and 'Item Name' get the values for the DataFrame 
item_purchase = purchase_data.groupby(['Item ID', 'Item Name'])['Price'].count()
total_item_pvalue = purchase_data.groupby(['Item ID', 'Item Name'])['Price'].sum()
item_price = total_item_pvalue/item_purchase

#creating the DataFrame
most_popular_item = pd.DataFrame({'Purchase Count': item_purchase,
                                 'Item Price': item_price,
                                 'Total Purchase Value': total_item_pvalue})
#formatting, all money data with $ and to sort DataFrame to display in descending order to get most purchased item on top
most_popular_item['Item Price'] = most_popular_item['Item Price'].map('${:.2f}'.format)
most_popular_item['Total Purchase Value'] = most_popular_item['Total Purchase Value'].map('${:.2f}'.format)
most_popular_item.sort_values(by=['Purchase Count'],ascending = False).head(5)


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [11]:
#sorting by Total Purchase Value 
most_profitable_item = most_popular_item.sort_values(by=['Purchase Count', 'Total Purchase Value'],ascending = False)
most_profitable_item.head(5)


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


The most sought after and most profitable in-game items are Final Critic and Oathbreaker, Last Hope of the Breaking Storm. Despite being higher priced, they are the most purchased in-game item.  

Hence, in order to further promote the game and increase the purchase transaction, the game can target Male players of the age range (15 to 29) and have a special discount on the top 3 most purchased in-game item. 
